In [1]:
# Thsi is the code used for the paper results
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import os
import sys
import numpy as np 
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct
import matplotlib.pyplot as plt


In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 10,
    "exp_name": ct.generate_readable_short_id("Paper_v1_feature_size"),
    "path": "../../",
    "split": "CV", #CV, random_split
    "cv_folds": 10,

    # "test_split": 0.1,
    # "calib_split": 0.11,
    
    "plot": False,
    "calib_methods": ["RF_d", "RF_opt", "RF_large",
                      "Platt", "ISO", "Beta", "CRF", "VA",
                      "tlr", "Rank", "Elkan",                      
                      ],
    
    "metrics": ["acc", "brier", "ece", "logloss", "tce"],

    #data
    "data_name": "synthetic",
    "plot_data": True,
    "data_size": 1000,
    "n_features": 2,

    "class1_mean_min":0, 
    "class1_mean_max":1,
    "class2_mean_min":0, 
    "class2_mean_max":1, 


    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth": np.arange(2, 100).tolist(),
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": np.arange(2, 11).tolist(),
                    "min_samples_leaf": np.arange(1, 11).tolist(),
                    # 'bootstrap': [True, False],                # Whether bootstrap samples are used
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False]
                    },
    
    "oob": False,
    "laplace": 0, # if set to 1, OOB must be false

    "exp_key": "n_features",
    "exp_values":  np.arange(2, 50, 1).tolist()
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

Success in 2 n_features
exp_param 2 done
Success in 3 n_features
exp_param 3 done
Success in 4 n_features
exp_param 4 done
Success in 5 n_features
exp_param 5 done
Success in 6 n_features
exp_param 6 done
Success in 7 n_features
exp_param 7 done
Success in 8 n_features
exp_param 8 done
Success in 9 n_features
exp_param 9 done
Success in 10 n_features
exp_param 10 done
Success in 11 n_features
exp_param 11 done
Success in 12 n_features
exp_param 12 done
Success in 13 n_features
exp_param 13 done
Success in 14 n_features
exp_param 14 done
Success in 15 n_features
exp_param 15 done
Success in 16 n_features
exp_param 16 done
Success in 17 n_features
exp_param 17 done
Success in 18 n_features
exp_param 18 done
Success in 19 n_features
exp_param 19 done
Success in 20 n_features
exp_param 20 done
Success in 21 n_features
exp_param 21 done
Success in 22 n_features
exp_param 22 done
Success in 23 n_features
exp_param 23 done
Success in 24 n_features
exp_param 24 done
Success in 25 n_features
ex

In [6]:
plot_calib_methods = params["calib_methods"].copy()
# plot_calib_methods.remove("RF_large")
plot_calib_methods.remove("Elkan")
# plot_calib_methods.remove("RF")
plot_calib_methods

['RF_d',
 'RF_opt',
 'RF_large',
 'Platt',
 'ISO',
 'Beta',
 'CRF',
 'VA',
 'tlr',
 'Rank']

In [7]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=False, 
                                    std=True)

path = f"results/{params['exp_name']}/features"
if not os.path.exists(path):
    os.makedirs(path)

for metric in params["metrics"]: #
    ax = tables[metric][plot_calib_methods].plot()
    ax.set_xlabel(params["exp_key"])
    ax.set_ylabel(metric)
    plt.savefig(f"{path}/f_{metric}.pdf", format='pdf', transparent=True)
    plt.close() 
